# Parameters

In [2]:
data_size = 2000; dataset_views_to_consider = 'all'; seed = 42
d_input_enc=2000; nb_classes_dec=33; class_weights=[]; d_model_enc_dec=1024; d_ff_enc_dec=1024; n_heads_enc_dec=16; n_layers_enc=2; n_layers_dec=2; activation="relu"; dropout=0.1 
model_params = {
    "d_input_enc": 2000, 
    "lr": 6.033193735866575e-05,
    "nb_classes_dec": 33,
    "early_stopping": True,
    "dropout": 0.16171970479206027,
    "weight_decay": 5.4598394312421854e-05,
    "activation": "relu",
    "optimizer": "Adam",
    "lr_scheduler": "cosine_with_restarts",
    "loss": "ce",
    "n_epochs": 1, 
    "batch_size": 256,
    "class_weights":[4.03557312, 0.85154295, 0.30184775, 1.18997669, 8.25050505,
            0.72372851, 7.73484848, 1.81996435, 0.62294082, 0.61468995,
            4.07992008, 0.49969411, 1.07615283, 1.85636364, 0.7018388 ,
            0.84765463, 0.60271547, 0.62398778, 4.26750261, 0.61878788,
            1.89424861, 1.98541565, 0.65595888, 2.05123054, 1.37001006,
            0.77509964, 0.76393565, 2.67102681, 0.64012539, 2.94660895,
            0.64012539, 6.51355662, 4.64090909],
    "d_model_enc_dec": 512,
    "n_heads_enc_dec": 16,
    "n_layers_enc": 10,
    "n_layers_dec": 1
}
d_ff_enc_dec_value = model_params["d_model_enc_dec"] * 4
model_params["d_ff_enc_dec"] = d_ff_enc_dec_value

fit_params = {
    "nb_ckpts":1, 
    "verbose":1
}

predict_params = {
    "nb_ckpts":1, 
    "scores_fname": "transformer_scores.json"
}

training_params = {
    "model_params": model_params,
    "fit_params": fit_params,
    "predict_params": predict_params,
    "data_size": int('2000'),
    "dataset_views_to_consider": 'all',
    "exp_type": "data_aug",
    "seed": 42
}
output_path = './'
# salloc --time=02:00:00 --nodes=1 --gres=gpu:1 --ntasks-per-node=32  --mem-per-cpu=127000M  --account=rrg-corbeilj-ac
# salloc --time=06:00:00 --gres=gpu:v100:1 --cpus-per-task=3 --mem=32000M --account=rrg-corbeilj-ac

# Import section

In [6]:
import argparse
import os
import json
import torch
import random
import natsort
import numpy as np
import pandas as pd
from tqdm import tqdm
from argparse import Namespace


import optuna
from optuna.study import StudyDirection
from packaging import version
from multiomic_modeling.models.trainer import MultiomicTrainer
from multiomic_modeling.models.trainer_multimodal import MultiomicTrainerMultiModal
from multiomic_modeling.models.base import BaseTrainer
from multiomic_modeling.data.data_loader import MultiomicDatasetDataAug, MultiomicDatasetNormal, MultiomicDatasetBuilder, SubsetRandomSampler
from multiomic_modeling.models.models import MultiomicPredictionModel, MultiomicPredictionModelMultiModal
from multiomic_modeling.models.utils import expt_params_formatter, c_collate
from multiomic_modeling.loss_and_metrics import ClfMetrics, NumpyEncoder
from multiomic_modeling.utilities import params_to_hash
from multiomic_modeling.torch_utils import to_numpy, totensor, get_optimizer
from multiomic_modeling import logging
from torch.utils.data import DataLoader
from transformers.optimization import Adafactor, AdamW, \
    get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

import pytorch_lightning as pl
import torch

if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")



In [17]:
dataset_views_to_consider='3_main_omics'
dataset = MultiomicDatasetNormal(data_size=data_size, views_to_consider=dataset_views_to_consider)
train, test, valid = MultiomicDatasetBuilder().multiomic_data_normal_builder(dataset=dataset, 
                                                                                 test_size=0.2, 
                                                                                 valid_size=0.1, 
                                                                                 random_state=seed)
dataset_augmented = MultiomicDatasetDataAug(train_dataset=train, data_size=data_size, views_to_consider=dataset_views_to_consider)
train_augmented = MultiomicDatasetBuilder.multiomic_data_aug_builder(augmented_dataset=dataset_augmented)
train_data_loader = MultiomicDatasetBuilder().multiomic_dataset_loader(dataset=train, batch_size=len(train))
train_augmented_data_loader = MultiomicDatasetBuilder().multiomic_dataset_loader(dataset=train_augmented, batch_size=len(train_augmented))
test_data_loader = MultiomicDatasetBuilder().multiomic_dataset_loader(dataset=test, batch_size=len(test))
valid_data_loader = MultiomicDatasetBuilder().multiomic_dataset_loader(dataset=valid, batch_size=len(valid))


In [19]:
train_dataset_array = next(iter(train_data_loader))[0][-1].numpy()
# next(iter(train_data_loader))[0][-1]

(8465, 3)

In [22]:
for el in train_dataset_array[:10]: 
    print(np.where(el == False))

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([0]),)


In [24]:
f = np.where(train_dataset_array == False)

In [28]:
f[1].shape

(2755,)

In [4]:
# print(len(train.indices))
# print(train[0])
# print(len(train[0]))
# print(len(train[0][0]))

# print(len(train_augmented.indices))
# print(train_augmented[0])
# print(len(train_augmented[0]))
# print(len(train_augmented[0][0]))

8465
((array([[ 0.974738,  0.947639,  0.966306, ...,  0.644351,  0.75878 ,
         0.148801],
       [ 5.93    , 16.6     ,  1.58    , ...,  0.      ,  0.      ,
         0.      ],
       [12.79    , 10.44    ,  0.      , ...,  7.01    ,  7.9     ,
         3.7     ]]), array([ True,  True,  True])), 2, 'TCGA-D8-A1XC-01')
3
2
25395
((array([[ 0.962224 ,  0.963145 ,  0.1115   , ...,  0.361789 ,  0.0362509,
         0.0450632],
       [ 3.64     ,  9.19     ,  3.04     , ...,  0.       ,  0.       ,
         0.       ],
       [12.96     ,  0.       ,  0.       , ...,  7.54     ,  6.62     ,
         4.68     ]]), array([ True,  True,  True]), array([[ 0.962224 ,  0.963145 ,  0.1115   , ...,  0.361789 ,  0.0362509,
         0.0450632],
       [ 3.64     ,  9.19     ,  3.04     , ...,  0.       ,  0.       ,
         0.       ],
       [12.96     ,  0.       ,  0.       , ...,  7.54     ,  6.62     ,
         4.68     ]]), array([ True,  True,  True])), 0, 'TCGA-OR-A5J2-01')
3
4


In [5]:
model = MultiomicTrainer.run_experiment(**training_params, output_path=output_path)

>>> Training configuration : 
{
  "data_size": 2000,
  "dataset_views_to_consider": "all",
  "exp_type": "data_aug",
  "fit_params": {
    "nb_ckpts": 1,
    "verbose": 1
  },
  "kwargs": {},
  "model_params": {
    "activation": "relu",
    "batch_size": 256,
    "class_weights": [
      4.03557312,
      0.85154295,
      0.30184775,
      1.18997669,
      8.25050505,
      0.72372851,
      7.73484848,
      1.81996435,
      0.62294082,
      0.61468995,
      4.07992008,
      0.49969411,
      1.07615283,
      1.85636364,
      0.7018388,
      0.84765463,
      0.60271547,
      0.62398778,
      4.26750261,
      0.61878788,
      1.89424861,
      1.98541565,
      0.65595888,
      2.05123054,
      1.37001006,
      0.77509964,
      0.76393565,
      2.67102681,
      0.64012539,
      2.94660895,
      0.64012539,
      6.51355662,
      4.64090909
    ],
    "d_ff_enc_dec": 2048,
    "d_input_enc": 2000,
    "d_model_enc_dec": 512,
    "dropout": 0.16171970479206027,
  

/home/maoss2/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Checkpoint directory ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name    | Type                     | Params
-----------------------------------------------------
0 | network | MultiomicPredictionModel | 36.8 M
-----------------------------------------------------
36.8 M    Trainable params
0         Non-trainable params
36.8 M    Total params
147.085   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/maoss2/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Process Process-8:
Process Process-6:
Process Process-7:
Process Process-5:
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x2b09f0b4ed30>
Traceback (most recent call last):
  File "/home/maoss2/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/maoss2/miniconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/maoss2/miniconda3/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/maoss2/miniconda3/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File

[INFO] Testing....
./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v1.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v2.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v3.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v4.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v5.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00-v6.ckpt ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints/epoch=0--val_loss=0.00.ckpt


RuntimeError: Error(s) in loading state_dict for MultiomicTrainer:
	Missing key(s) in state_dict: "network._MultiomicPredictionModel__loss.weight". 
	Unexpected key(s) in state_dict: "network.decoder_views.decoder.layers.0.self_attn.in_proj_weight", "network.decoder_views.decoder.layers.0.self_attn.in_proj_bias", "network.decoder_views.decoder.layers.0.self_attn.out_proj.weight", "network.decoder_views.decoder.layers.0.self_attn.out_proj.bias", "network.decoder_views.decoder.layers.0.multihead_attn.in_proj_weight", "network.decoder_views.decoder.layers.0.multihead_attn.in_proj_bias", "network.decoder_views.decoder.layers.0.multihead_attn.out_proj.weight", "network.decoder_views.decoder.layers.0.multihead_attn.out_proj.bias", "network.decoder_views.decoder.layers.0.linear1.weight", "network.decoder_views.decoder.layers.0.linear1.bias", "network.decoder_views.decoder.layers.0.linear2.weight", "network.decoder_views.decoder.layers.0.linear2.bias", "network.decoder_views.decoder.layers.0.n

In [ ]:
torch.cuda.is_available()